<a href="https://colab.research.google.com/github/mengjie514/Moral-Emotional-Analysis/blob/main/HPALL_SampleCode_I_Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/Colab Notebooks/PSIV")
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)

import json

data_list = []
tweets_info_temp = {
    'Corpus':'',
    'tweet_id':0,
    'tweet_text':'',
    'annotation_0':'',
    'annotation_1':'',
    'annotation_2':'',
    'annotation_3':'',
    'annotation_4':'',
    'annotation_5':'',
    'annotation_6':'',
    'annotation_7':'',
}

with open('MFTC_V4_text.json','r') as f:
    data = json.loads(f.read())

    for corpus_index in range(len(data)):
        Tweets = data[corpus_index]['Tweets']

        for tweet in Tweets:
            tweet_info = tweets_info_temp.copy()
            tweet_info['Corpus'] = data[corpus_index]['Corpus']
            tweet_info['tweet_id'] = tweet['tweet_id']
            tweet_info['tweet_text'] = tweet['tweet_text']

            annotation_count = 0
            for item in tweet['annotations']:
                tweet_info[f'annotation_{annotation_count}'] = item['annotation']
                annotation_count += 1

            data_list.append(tweet_info)

data_list[0:1]

Mounted at /content/gdrive


<ipython-input-1-9ebe1b0ea74c>:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


[{'Corpus': 'ALM',
  'tweet_id': '521033092132503552',
  'tweet_text': '@fergusonoctober @FOX2now #AllLivesMatter Peace and Love Prevail. God Bless.\n',
  'annotation_0': 'care',
  'annotation_1': 'care,purity',
  'annotation_2': 'care,purity',
  'annotation_3': 'care',
  'annotation_4': '',
  'annotation_5': '',
  'annotation_6': '',
  'annotation_7': ''}]

**1. Annotation Agreement**

In [ ]:
# transfer data_list to dataframe
df = pd.DataFrame(data_list)

# annotation agreement
df1 = df.replace(r'^\s*$', np.nan, regex=True)

df1['annotation_n'] = df1.loc[:, 'annotation_0':'annotation_7'].apply(lambda x: x.notna().sum(), axis=1)

# count for each type of annotation
df1['annotation_all'] = df1[df1.columns[3:11]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

words = ['care', 'harm', 'fairness', 'cheating', 'loyalty','betrayal','authority', 'subversion', 'purity', 'degradation', 'non-moral']
df2 = df1.annotation_all.str.extractall('({})'.format('|'.join(words)))\
                           .iloc[:, 0].str.get_dummies().sum(level=0)

df = pd.merge(df1, df2, left_index=True, right_index=True)

# major annotation(s)
df.loc[:,'authority':'subversion'] = df.loc[:,'authority':'subversion'].div(df.annotation_n,0)

# as df.ge should running with int, using sub_df to extract int columns
sub_df = df.loc[:,'authority':'subversion']
df3 = pd.DataFrame(sub_df.ge(0.501).T.agg(lambda x: sub_df.columns[x].tolist()))
df3.rename(columns={0:'annotation_main'},inplace=True)
df3.annotation_main= df3.annotation_main.str.join(',')

df = pd.merge(df, df3, left_index=True, right_index=True)
df.head(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  app.launch_new_instance()


,Corpus,tweet_id,tweet_text,annotation_0,annotation_1,annotation_2,annotation_3,annotation_4,annotation_5,annotation_6,...,care,cheating,degradation,fairness,harm,loyalty,non-moral,purity,subversion,annotation_main
0,ALM,521033092132503552,@fergusonoctober @FOX2now #AllLivesMatter Peace and Love Prevail. God Bless.\n,care,"care,purity","care,purity",care,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,care


**2. Text Pre-processing**

In [ ]:
# text Cleaning
import string
from html.parser import HTMLParser
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer, word_tokenize

# hashtags check
hash_list = df.tweet_text.str.extractall(
    r'(\#\w+)'
).reset_index().drop_duplicates(['level_0', 0])[0].value_counts()

split_adhere_dic = {
    "weve": "we have", "Weve": "We have",
    "youve": "you have", "Youve": "You have",
    "theyve": "they have", "Theyve": "They have",
    "Ive": "I have",
    "youre": "you are", "Youre": "You are",
    "theyre": "they are", "Theyre": "They are",
    "Im": "I am", "Ill": "I will", "Id": "I would",
    "hes": "he's", "Hes": "He's",
    "shes": "she's", "Shes": "She's",
    "thats": "that is", "Thats": "That is",
    "thatd": "that would", "Thatd": "That would",
    "theres": "there is", "Theres": "There is",
    "thered": "there would", "Thered": "There would",
    "heres": "here is", "Heres": "Here is",
    # negation cues
    "isn't": "is not", "isnt": "is not",
    "ain't": "are not", "aint": "are not",
    "aren't": "are not", "arent": "are not",
    "weren't": "were not", "werent": "were not",
    "wasn't": "was not", "wasnt": " was not",
    "cannot": "can not", "can't": " can not", "cant": " can not",
    "couldn't": "could not", "couldnt": " could not",
    "wouldn't": "would not", "wouldnt": "would not",
    "don't": "do not", "dont": "do not",
    "doesn't": "does not", "doesnt": "does not",
    "didn't": "did not", "didnt": "did not",
    "haven't": "have not", "havent": "have not", "havnt": "have not",
    "hasn't": "has not", "hasnt": "has not",
    "hadn't": "had not", "hadnt": "had not",
    "shouldn't": "should not", "shouldnt": "should not",
    "won't": "will not", "wont": "will not",
    "mightn't": "might not", "mightnt": "might not",
    "mustn't": "must not", "mustnt": "must not",
    "needn't": "need not", "neednt": "need not"
    }

pat1 = r'@[\w_]+'
pat2 = r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+'
pat3 = r'#'
pat3b = r'#.'
pat4 = r'www.[^ ]+'
pat5 = r'\n'
pat6= r'\xa0'
pat7= r'AT_USER'
pat8 = r'pic.twitter.com/(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+'
pat9 = r'.twitter.com/(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+'
combined_pat = r'|'.join((pat1, pat2, pat3, pat3b, pat4, pat5, pat6, pat7, pat8, pat9))

split_pattern = re.compile(r'\b(' + '|'.join(split_adhere_dic.keys()) + r')\b')

def tweet_cleaner(demo):
    soup = BeautifulSoup(demo, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    splitted = re.sub(r"([a-z\.!?])([A-Z])", r"\1 \2", stripped)
    split_handled = split_pattern.sub(lambda x: split_adhere_dic[x.group()], splitted)

    return split_handled

df['new_clean_text'] = [tweet_cleaner(t) for t in df.tweet_text]

# Remove stopwords/punctuations/numbers
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Add words that aren't in the NLTK stopwords list and are frequently used in MFT
new_stopwords = ['could', 'would', 'day', 'days', 'month', 'months', 'year', 'years',
                 'say', 'says', 'said', 'saying', 'gotta', 'wanna', 'etc']
new_stopwords_list = stop_words.union(new_stopwords)

# Negation Scope Detection for Twitter Sentiment Analysis (Reitan et al, 2015)
not_stopwords = {'no', 'not', 'nor', 'none', 'neither', 'never', 'nothing'}
final_stop_words = set([word for word in new_stopwords_list if word not in not_stopwords])

table = str.maketrans('', '', string.punctuation)

def text_process(deep):
    words = re.split(r'\W+', deep)
    nopunc = [w.translate(table) for w in words]
    nostop =  ' '.join([word for word in nopunc if word.lower() not in final_stop_words])
    nonum = ' '.join([word for word in str(nostop).split() if not word.isdigit()])
    return nonum

df['new_clean_text_deep'] = df.apply(lambda row: text_process(row.new_clean_text), axis=1)

# Stemming words
from nltk.stem import PorterStemmer, WordNetLemmatizer
porter_stemmer = PorterStemmer()

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df['new_clean_text_deep_stem'] = df['new_clean_text_deep'].apply(stem_sentences)
df.head(1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Corpus,tweet_id,tweet_text,annotation_0,annotation_1,annotation_2,annotation_3,annotation_4,annotation_5,annotation_6,...,fairness,harm,loyalty,non-moral,purity,subversion,annotation_main,new_clean_text,new_clean_text_deep,new_clean_text_deep_stem
0,ALM,521033092132503552,@fergusonoctober @FOX2now #AllLivesMatter Peace and Love Prevail. God Bless.\n,care,"care,purity","care,purity",care,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.5,0.0,care,All Lives Matter Peace and Love Prevail. God Bless.,Lives Matter Peace Love Prevail God Bless,live matter peac love prevail god bless


In [ ]:
# remove substring which conains numbers
df.new_clean_text_deep_stem = df.new_clean_text_deep_stem.str.replace('[\w_]+\d+[\w_]+', ' ')
df.new_clean_text_deep_stem = df.new_clean_text_deep_stem.str.replace('\d+[\w_]+', ' ')
df.new_clean_text_deep_stem = df.new_clean_text_deep_stem.str.replace('[\w_]+\d+', ' ')

l = ["sandi", "nyc", " NY ", "blm", " url", " http", " htt ", " via", " ppl ", " pic ", "baltimor",
     " Dr ", " Mr ", " Sr ", " AL ", " TL ", " CA ", " AP ", " AC ", " PD ", " NJ ", " JK ",
     " MC ", " AR ", " PA ", " KU ", " AG ", " NE ", " MN ", " SE ", " BM ", " BW ", " EI ",
     " DC ", " OT ", " GO ", " LA ", " CK ", " EC ", " SA ", " MT ", " BS ", " DE ", " KS ",
     " QB ", " LE ", " OA ", " PA ", " DJ ", " DM ", " WH ", " OI ", " ST ", " AG ", " SC ",
     " GT ", " MS ", " CD ", " VO ", " BI ", " ET ", " AH ", " NW ", " RP ", " Go ", " CE ",
     " Oh ", " OH ", " FL ", " BA ", " FU ", " YO ", " GS ", " BO ", " BC ", " VA ", " BT ",
     " LX ", " DA ", " EN ", " HT ", " AW ", " Co ", " PD ", " SA ", " OD ", " NE ", " XI ",
     " u ", " c ", " d ", " a ", " b ", " t ", " k ", " m ", " v ", " n ", " r ", " w ",
     " aa ", " cc ", " kkk", " ur ", " fr ", " ya ", " tf ", " fb ", " stfu ",
     " N ", " F ", " K ", " B ", " J ", " Z ", " H ", " R ", " U ", " E ", " P ", " W ", " Q ", " G ", " L ", " I ",
     "uddfa", "uddef", "uddeb", "udffb", "udffd", "udffb", "udffc", "udde", "udcaf", "udcaaladi", "udcfa", "uddf", "udcaa",
     " th ", " nd ", " am ", " pm ", " kg ", " hr ", "et al",
     "Ù", "Ü", "Û", "û", "û", "â", "å", "è", "Ç", "Ï", "ï", "ā", "ª", "Ó", "Ð", "Õ", "ù"]

df.new_clean_text_deep_stem = df.new_clean_text_deep_stem.str.replace('|'.join(l), ' ', regex=True).str.strip().replace(' ', '')

#df.to_excel('output_for_R_050_use.xlsx')